In [1]:
import numpy as np

In [16]:
def compute_gradient_2d(E, dy, dx):
    # Use second-order central differencing in the interior, and forward/backward at the boundaries
    dEdy = np.zeros_like(E)
    dEdx = np.zeros_like(E)
    
    # Central differencing in the interior (higher order)
    dEdy[1:-1, :] = (E[2:, :] - E[:-2, :]) / (dy[1:-1, :] + dy[:-2, :])
    dEdx[:, 1:-1] = (E[:, 2:] - E[:, :-2]) / (dx[:, 1:-1] + dx[:, :-2])

    # Forward differencing at the first row (North boundary)
    dEdy[0, :] = (E[1, :] - E[0, :]) / dy[0, :]
    # Backward differencing at the last row (South boundary)
    dEdy[-1, :] = (E[-1, :] - E[-2, :]) / dy[-1, :]

    # Forward differencing at the first column (Prime meridian)
    dEdx[:, 0] = (E[:, 1] - E[:, 0]) / dx[:, 0]
    # Backward differencing at the last column (International Date Line)
    dEdx[:, -1] = (E[:, -1] - E[:, -2]) / dx[:, -1]
    
    return dEdy, dEdx

def estimate_E_from_gradients_high_order(dEdy, dEdx, dy, dx):
    ny, nx = dEdy.shape
    E_estimated = np.zeros((ny, nx))

    # Integrate dEdy along y (downward integration)
    E_estimated[1:, 0] = np.cumsum(dEdy[:-1, 0] * dy[:-1, 0])

    # Integrate dEdx along x for each row
    for i in range(ny):
        E_estimated[i, 1:] = np.cumsum(dEdx[i, :-1] * dx[i, :-1])

    return E_estimated

# Define the grid
ny, nx = 25, 50
ly, lx = 1.0, 1.0

# Create grid
x = np.linspace(0, lx, nx)
y = np.linspace(0, ly, ny)
X, Y = np.meshgrid(x, y)

# Calculate dx and dy
dx = np.gradient(X, axis=1)
dy = np.gradient(Y, axis=0)

# Define the source term f (Gaussian distribution)
sigma = 0.1
f = np.exp(-((X - lx/2)**2 + (Y - ly/2)**2) / (2 * sigma**2))

# Compute the numerical gradient using compute_gradient_2d
dfdy, dfdx = compute_gradient_2d(f, dy, dx)

f_recover = estimate_E_from_gradients(dfdy, dfdx, dy, dx)

error = np.mean(np.abs(f_recover - f))

print('error: {}'.format(error))

error: 0.004787738236913826


In [8]:
# Calculate the errors between the numerical and exact derivatives
error_dfdx = np.abs(dfdx_numeric - dfdx_exact)
error_dfdy = np.abs(dfdy_numeric - dfdy_exact)

max_error_dfdx = np.max(error_dfdx)
max_error_dfdy = np.max(error_dfdy)

max_error_dfdx, max_error_dfdy

(0.9999910129439605, 0.9994682903047929)